In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from pyaw import utils_preprocess
from pyaw.swarm import save_SW_EFIx_TCT16, save_SW_MAGx_HR_1B

In [2]:
start_time = '20160311T064640'
end_time = '20160311T064920'
satellite = 'A'
sdir = Path("./data")
sfn_EFIA_TCT16 = Path("EFIA_TCT16_20160311T064640_20160311T064920.pkl")
sfn_MAGA_HR_1B = Path("MAGA_HR_1B_20160311T064640_20160311T064920.pkl")

In [3]:
save_SW_EFIx_TCT16(start_time, end_time, satellite, sdir,
                   sfn_EFIA_TCT16)
save_SW_MAGx_HR_1B(start_time, end_time, satellite, sdir,
                   sfn_MAGA_HR_1B)

data\EFIA_TCT16_20160311T064640_20160311T064920.pkl already exists, skip save.
data\MAGA_HR_1B_20160311T064640_20160311T064920.pkl already exists, skip save.


# read data

In [4]:
df_e = pd.read_pickle(Path(sdir) / Path(sfn_EFIA_TCT16))
df_b = pd.read_pickle(Path(sdir) / Path(sfn_MAGA_HR_1B))
timestamps_e = df_e.index.values
timestamps_b = df_b.index.values
del sdir,sfn_EFIA_TCT16,sfn_MAGA_HR_1B

# to verify the correlation between the satellite-track coordinate system and the S/C coordinate system

## IGRF13

NEC coordinate system

In [5]:
latitudes = df_e['Latitude'].values
longitudes = df_e['Longitude'].values
radiuses = df_e['Radius'] / 1e3  # km

In [6]:
from igrf13 import igrf_utils
# from importlib import reload
# reload(igrf_utils)

igrf_bn,igrf_be,igrf_bc = igrf_utils.get_igrf13_b_nec(timestamps_e, latitudes, longitudes, radiuses, igrf13shc_path="../igrf13/IGRF13.shc")

the whole loop takes time: 7.070488691329956s


### do rotation

from NEC coordinate system to S/C coordinate system

In [7]:
VsatN = df_e['VsatN'].values
VsatE = df_e['VsatE'].values
rotation_matrix_2d_nec2sc,rotation_matrix_2d_sc2nec = utils_preprocess.get_rotation_matrices_nec2sc_sc2nec(VsatN,VsatE)
igrf_b_sc_x,igrf_b_sc_y = utils_preprocess.do_rotation(igrf_bn, igrf_be, rotation_matrix_2d_nec2sc)

### compare IGRF magnetic field in S/C coordinate system with the magnetic field in the satellite-track coordinate system

In [8]:
Bx = df_e['Bx'].values
By = df_e['By'].values
Bz = df_e['Bz'].values

In [9]:
igrf_b_sc_x,Bx

(array([5488.84597893, 5490.36120071, 5491.89705865, ..., 9822.42766948,
        9824.24009966, 9826.02530818]),
 array([-5522.3447, -5523.882 , -5525.445 , ..., -9703.592 , -9704.914 ,
        -9706.217 ], dtype=float32))

In [10]:
igrf_b_sc_y,By

(array([2402.89668432, 2403.29288557, 2403.69322004, ..., 3269.95250741,
        3270.22039535, 3270.48448059]),
 array([-2376.3591, -2376.8003, -2377.2485, ..., -3364.922 , -3365.4243,
        -3365.9182], dtype=float32))

In [11]:
np.array(igrf_bc),Bz

(array([47135.03779735, 47134.75750318, 47134.47364147, ...,
        45397.81913387, 45396.67775593, 45395.55296498]),
 array([47170.96 , 47170.67 , 47170.383, ..., 45423.27 , 45422.24 ,
        45421.223], dtype=float32))

### compare the Bx,By,Bz of EFIx_TCT16 download from VirES with EFIx_TCT16 download from ESA Swarm http database

In [12]:
import cdflib

fp = "./data/SW_EXPT_EFIA_TCT16_20160311T000000_20160311T065505_0302.cdf"

cdf_file = cdflib.CDF(fp)
info = cdf_file.cdf_info()

In [13]:
cdf_file.varattsget('Bx')

{'FIELDNAM': 'Bx',
 'CATDESC': 'Geomagnetic field x component in satellite-track coordinates, derived from the 1 Hz product.',
 'Type': 'CDF_FLOAT',
 'UNITS': 'nT',
 'VAR_TYPE': 'data',
 'DEPEND_0': 'Time',
 'DISPLAY_TYPE': 'time_series',
 'LABLAXIS': 'Bx',
 'VALIDMIN': -65000.0,
 'VALIDMAX': 65000.0}

In [14]:
Bx_local = cdf_file.varget('Bx')
timestamps_local = cdf_file.varget('Timestamp')

In [15]:
import numpy as np

# Create a numpy datetime object for Year 0, January 1, and the unit is millisecond
ad0_reference = np.datetime64('0000-01-01', 'ms')

# Convert the 'Timestamp' to datetime by adding the milliseconds since the epoch_reference
# Assuming 'Timestamp' is in milliseconds
timestamps_local = ad0_reference + np.array(timestamps_local, dtype='timedelta64[ms]')

# Print the first few datetime values
print(timestamps_local[:5])

['2016-03-11T00:00:00.006' '2016-03-11T00:00:00.068'
 '2016-03-11T00:00:00.131' '2016-03-11T00:00:00.193'
 '2016-03-11T00:00:00.256']


In [16]:
mask = np.where(timestamps_local > np.datetime64('2016-03-11T06:47:00'))
Bx_local = Bx_local[mask]
Bx_local

array([ -6025.4023,  -6026.9688,  -6028.56  , ..., -18554.49  ,
       -18555.754 , -18556.998 ], dtype=float32)

### conclusion

(x,y,z) in satellite-track coordinate system, in S/C coordinate system, it will be (-x,-y,z)

In [17]:
# todo:: according to the doc of tct16, the x axis of the satellite-track coordinate system is in the direction of the satellite velocity that is same to the x axis of the S/C coordinate system??? so why there is a negative sign difference between the x and y components of the magnetic field in the satellite-track coordinate system and the S/C coordinate system?